# Hydro Availability Analysis

- Calculate the seasonal availability from the monthly data
- Format both HydroReservoir and Run-of-River (ROR) plants for EPM

In [51]:
import os
import pandas as pd

## 0. User Input

In [52]:
# Define the path to the input data
filename = 'hydro_profile_dry.csv'

# Define the seasonal distribution of months
seasons_dict = {
    1: 2,
    2: 2,
    3: 2,
    4: 2,
    5: 1,
    6: 1,
    7: 1,
    8: 1,
    9: 1,
    10: 2,
    11: 2,
    12: 2
}

#
folder_epm = os.path.join('..', '..', 'epm', 'input', 'data_capp')
if not os.path.exists(folder_epm):
    raise FileNotFoundError(f"EPM input folder not found: {os.path.abspath(folder_epm)}")
else:
    template_path = os.path.join(folder_epm, 'pHours.csv')
    if not os.path.exists(template_path):
        raise FileNotFoundError(f"The file {os.path.abspath(template_path)} does not exist. Please check the path.")

template = pd.read_csv(template_path, index_col=[0, 1], header=0)
print("Template loaded successfully. Proceeding with the analysis.")
display(template.head())


Template loaded successfully. Proceeding with the analysis.


t1     t2     t3     t4     t5     t6     t7     t8     t9  \
season daytype                                                                  
Q1     d1         2.0    2.0    2.0    2.0    2.0    2.0    2.0    2.0    2.0   
       d2         5.0    5.0    5.0    5.0    5.0    5.0    5.0    5.0    5.0   
       d3         6.0    6.0    6.0    6.0    6.0    6.0    6.0    6.0    6.0   
       d4       105.0  105.0  105.0  105.0  105.0  105.0  105.0  105.0  105.0   
       d5        35.0   35.0   35.0   35.0   35.0   35.0   35.0   35.0   35.0   

                  t10  ...    t15    t16    t17    t18    t19    t20    t21  \
season daytype         ...                                                    
Q1     d1         2.0  ...    2.0    2.0    2.0    2.0    2.0    2.0    2.0   
       d2         5.0  ...    5.0    5.0    5.0    5.0    5.0    5.0    5.0   
       d3         6.0  ...    6.0    6.0    6.0    6.0    6.0    6.0    6.0   
       d4       105.0  ...  105.0  105.0  105.0  105.0  105.0  105.0  105.0   
       d5        35.0  ...   35.0   35.0   35.0   35.0   35.0   35.0   35.0   

                  t22    t23    t24  
season daytype                       
Q1     d1         2.0    2.0    2.0  
       d2         5.0    5.0    5.0  
       d3         6.0    6.0    6.0  
       d4       105.0  105.0  105.0  
       d5        35.0   35.0   35.0  

[5 rows x 24 columns]

## 1. Folder Setup

In [53]:
folder_input = 'input'
folder_output = 'output'
if not os.path.exists(folder_output):
    os.makedirs(folder_output)
    print(f"Created output folder: {folder_output}")

## 2. Load Data

In [54]:
if not os.path.exists(os.path.join(folder_input, filename)):
    raise FileNotFoundError(f"The file {os.path.join(folder_input, filename)} does not exist. Please check the path.")
else:
    print(f"File {filename} found. Proceeding with the analysis.")
    data = pd.read_csv(os.path.join(folder_input, filename), index_col=None, header=[0])

File hydro_profile_dry.csv found. Proceeding with the analysis.


## 3. Process Data

### Reservoir Hydropower Plants

In [55]:
data_reservoir = data[data['tech'] == 'ReservoirHydro'].copy()
data_reservoir.set_index(['gen'], inplace=True)
data_reservoir.drop(columns=['zone', 'tech'], inplace=True)
# Ensure columns are int
data_reservoir.columns = data_reservoir.columns.astype(int)
display(data_reservoir.head())

# Groupby season_dict and calculate the mean for each season
data_reservoir = data_reservoir.T.groupby(seasons_dict).mean().T

# Add 'Q{}' prefix to the column names
data_reservoir.columns = [f'Q{col}' for col in data_reservoir.columns]
data_reservoir.columns.names = ['season']
display(data_reservoir.head())

data_reservoir.to_csv(os.path.join(folder_output, 'pAvailabilityCustom.csv'))
print(f"Reservoir data processed and saved to {os.path.join(folder_output, 'pAvailabilityCustom.csv')}")

,1,2,3,4,5,6,7,8,9,10,11,12
gen,,,,,,,,,,,,
AH Mabubas,0.570000,0.480000,0.410000,0.460000,0.550000,0.610000,0.780000,0.920000,1.000000,1.000000,0.950000,0.530000
Baynes,0.448571,0.401429,0.382857,0.408571,0.468571,0.542857,0.632857,0.672857,0.712857,0.738571,0.641429,0.478571
Bikongo,0.484167,0.502500,0.500000,0.504167,0.458333,0.310000,0.247500,0.225000,0.279167,0.334167,0.520000,0.559167
Boali 1 Rebuild,0.540000,0.550000,0.510000,0.490000,0.400000,0.240000,0.170000,0.140000,0.130000,0.180000,0.490000,0.620000
Boali 2,0.540000,0.550000,0.510000,0.490000,0.400000,0.240000,0.170000,0.140000,0.130000,0.180000,0.490000,0.620000


season,Q1,Q2
gen,,
AH Mabubas,0.772,0.628571
Baynes,0.606,0.500000
Bikongo,0.304,0.486310
Boali 1 Rebuild,0.216,0.482857
Boali 2,0.216,0.482857


Reservoir data processed and saved to output/pAvailabilityCustom.csv


### Run-of-River (ROR) Plants

In [56]:
def generate_pVREgenProfile(result, template):
    """Generate pVREgenProfile for ROR plants.

    This function takes a scenario identifier and a template DataFrame,
    and generates a pVREgenProfile DataFrame for Run-of-River (ROR) plants.
    It reshapes the result DataFrame to match the template structure,
    broadcasting the values across all hour columns.

    Parameters:
        result: pd.DataFrame
            The result DataFrame containing the seasonal data for ROR plants. The DataFrame should have 'gen' as the index and seasons as columns.
        template: pd.DataFrame
            The template DataFrame that defines the structure of the output.
    """

    result_reset = result.reset_index()  # so 'gen' becomes a column
    result_long = result_reset.melt(id_vars='gen', var_name='season', value_name='value')

    daytypes = template.reset_index()[['season', 'daytype']].drop_duplicates()
    merged = result_long.merge(daytypes, on='season', how='left')

    # get hour columns from template
    hour_cols = template.columns.difference(['season', 'daytype'])

    # broadcast the value across all hour columns
    for col in hour_cols:
        merged[col] = merged['value']

    merged_final = merged.drop(columns=['value'])
    merged_final = merged_final.set_index(['gen', 'season', 'daytype'])
    merged_final.index.names = ['gen', 'q', 'd']

    return merged_final

In [57]:
data_ror = data[data['tech'] == 'ROR'].copy()
data_ror.set_index(['gen'], inplace=True)
data_ror.drop(columns=['zone', 'tech'], inplace=True)
# Ensure columns are int
data_ror.columns = data_ror.columns.astype(int)
display(data_ror.head())
# Groupby season_dict and calculate the mean for each season
data_ror = data_ror.T.groupby(seasons_dict).mean().T
display(data_ror.head())

# Add 'Q{}' prefix to the column names
data_ror.columns = [f'Q{col}' for col in data_ror.columns]
data_ror.columns.names = ['season']

#
data_ror = generate_pVREgenProfile(data_ror, template)
data_ror = data_ror[template.columns]

display(data_ror.head())
data_ror.to_csv(os.path.join(folder_output, 'pVREgenProfile.csv'))

,1,2,3,4,5,6,7,8,9,10,11,12
gen,,,,,,,,,,,,
Akieni,0.570000,0.480000,0.410000,0.460000,0.550000,0.610000,0.780000,0.920000,1.000000,1.000000,0.950000,0.530000
Angouma,0.448571,0.401429,0.382857,0.408571,0.468571,0.542857,0.632857,0.672857,0.712857,0.738571,0.641429,0.478571
Baidou,0.464444,0.357778,0.387778,0.536667,0.558889,0.455556,0.237778,0.175556,0.314444,0.564444,0.716667,0.678889
Bayomen,0.364000,0.401000,0.439000,0.495000,0.443000,0.261000,0.226000,0.222000,0.244000,0.271000,0.352000,0.342000
Bihongore,0.464444,0.357778,0.387778,0.536667,0.558889,0.455556,0.237778,0.175556,0.314444,0.564444,0.716667,0.678889


,1,2
gen,,
Akieni,0.772000,0.628571
Angouma,0.606000,0.500000
Baidou,0.348444,0.529524
Bayomen,0.279200,0.380571
Bihongore,0.348444,0.529524


t1     t2     t3     t4     t5     t6     t7     t8     t9  \
gen    q  d                                                                   
Akieni Q1 d1  0.772  0.772  0.772  0.772  0.772  0.772  0.772  0.772  0.772   
          d2  0.772  0.772  0.772  0.772  0.772  0.772  0.772  0.772  0.772   
          d3  0.772  0.772  0.772  0.772  0.772  0.772  0.772  0.772  0.772   
          d4  0.772  0.772  0.772  0.772  0.772  0.772  0.772  0.772  0.772   
          d5  0.772  0.772  0.772  0.772  0.772  0.772  0.772  0.772  0.772   

                t10  ...    t15    t16    t17    t18    t19    t20    t21  \
gen    q  d          ...                                                    
Akieni Q1 d1  0.772  ...  0.772  0.772  0.772  0.772  0.772  0.772  0.772   
          d2  0.772  ...  0.772  0.772  0.772  0.772  0.772  0.772  0.772   
          d3  0.772  ...  0.772  0.772  0.772  0.772  0.772  0.772  0.772   
          d4  0.772  ...  0.772  0.772  0.772  0.772  0.772  0.772  0.772   
          d5  0.772  ...  0.772  0.772  0.772  0.772  0.772  0.772  0.772   

                t22    t23    t24  
gen    q  d                        
Akieni Q1 d1  0.772  0.772  0.772  
          d2  0.772  0.772  0.772  
          d3  0.772  0.772  0.772  
          d4  0.772  0.772  0.772  
          d5  0.772  0.772  0.772  

[5 rows x 24 columns]